## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Faya,SA,18.3851,42.4509,84.72,32,54,7.61,broken clouds
1,1,Meulaboh,ID,4.1363,96.1285,83.89,67,93,7.72,overcast clouds
2,2,Kinston,US,35.2627,-77.5816,71.78,94,0,0.00,mist
3,3,Pisco,PE,-13.7000,-76.2167,55.45,88,100,0.00,mist
4,4,Henties Bay,NaN,-22.1160,14.2845,59.90,72,9,3.94,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Faya,SA,18.3851,42.4509,84.72,32,54,7.61,broken clouds
1,1,Meulaboh,ID,4.1363,96.1285,83.89,67,93,7.72,overcast clouds
5,5,Cabo San Lucas,MX,22.8909,-109.9124,82.51,69,7,3.00,clear sky
7,7,Teluk Intan,MY,4.0259,101.0213,87.87,60,100,8.21,overcast clouds
9,9,Hilo,US,19.7297,-155.0900,79.29,97,100,3.44,overcast clouds
10,10,Dingle,PH,10.9995,122.6711,84.29,78,100,3.24,overcast clouds
12,12,Nikolskoye,RU,59.7035,30.7861,79.92,39,71,9.53,broken clouds
20,20,Bambous Virieux,MU,-20.3428,57.7575,75.45,53,75,16.11,broken clouds
21,21,Danshui,CN,22.7984,114.4672,88.68,65,100,15.21,light rain
24,24,Tessalit,ML,20.1986,1.0114,88.97,37,58,18.75,broken clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                246
City                   246
Country                245
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                245
City                   245
Country                245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Faya,SA,84.72,broken clouds,18.3851,42.4509,
1,Meulaboh,ID,83.89,overcast clouds,4.1363,96.1285,
5,Cabo San Lucas,MX,82.51,clear sky,22.8909,-109.9124,
7,Teluk Intan,MY,87.87,overcast clouds,4.0259,101.0213,
9,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,
10,Dingle,PH,84.29,overcast clouds,10.9995,122.6711,
12,Nikolskoye,RU,79.92,broken clouds,59.7035,30.7861,
20,Bambous Virieux,MU,75.45,broken clouds,-20.3428,57.7575,
21,Danshui,CN,88.68,light rain,22.7984,114.4672,
24,Tessalit,ML,88.97,broken clouds,20.1986,1.0114,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace('', np.nan)
clean_hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Faya,SA,84.72,broken clouds,18.3851,42.4509,املاك خالد الشعشوع
1,Meulaboh,ID,83.89,overcast clouds,4.1363,96.1285,Kost Pria
5,Cabo San Lucas,MX,82.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
7,Teluk Intan,MY,87.87,overcast clouds,4.0259,101.0213,Angsoka Hotel Teluk Intan
9,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Dingle,PH,84.29,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
12,Nikolskoye,RU,79.92,broken clouds,59.7035,30.7861,Tourist House - Sablino
20,Bambous Virieux,MU,75.45,broken clouds,-20.3428,57.7575,Casa Tia Villa
21,Danshui,CN,88.68,light rain,22.7984,114.4672,Grand Skylight International Hotel Huizhou
26,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [35]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))